In [1]:
import torch
import os
import numpy as np
import pandas as pd
import json

In [2]:
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import Descriptors
import sascorer
import sys
#from jtnn import sascorer
#from jtnn.mol_tree import Vocab
sys.path.append("./jtnn")
print(sys.path)

from jtnn import *

['', '/anaconda3/envs/my-rdkit-env/lib/python36.zip', '/anaconda3/envs/my-rdkit-env/lib/python3.6', '/anaconda3/envs/my-rdkit-env/lib/python3.6/lib-dynload', '/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages', '/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/IPython/extensions', '/Users/evanpike/.ipython', './jtnn']


In [3]:
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

test_path = "./data/opt.test.logP-SA"
vocab_path = "./data/vocab.txt"
model_path = "./joint-h420-L56-d3-noKL/model.iter-2"#"./joint-h420-L56-d3-beta0.001/model.iter-4"
hidden_size = 200
latent_size = 56
depth = 3
sim_cutoff = 0.0

# parser = OptionParser()
# parser.add_option("-t", "--test", dest="test_path")
# parser.add_option("-v", "--vocab", dest="vocab_path")
# parser.add_option("-m", "--model", dest="model_path")
# parser.add_option("-w", "--hidden", dest="hidden_size", default=200)
# parser.add_option("-l", "--latent", dest="latent_size", default=56)
# parser.add_option("-d", "--depth", dest="depth", default=3)
# parser.add_option("-s", "--sim", dest="cutoff", default=0.0)
# opts,args = parser.parse_args()

In [5]:
vocab = [x.strip("\r\n ") for x in open(vocab_path)] 
vocab = Vocab(vocab)

model = JTPropVAE(vocab, int(hidden_size), int(latent_size), int(depth))
model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))
#model = model.cuda()

data = []
with open(test_path) as f:
    for line in f:
        s = line.strip("\r\n ").split()[0]
        data.append(s)

res = []
for smiles in data:
    mol = Chem.MolFromSmiles(smiles)
    score = Descriptors.MolLogP(mol) - sascorer.calculateScore(mol)

    new_smiles,sim = model.optimize(smiles, sim_cutoff=sim_cutoff, lr=2, num_iter=80)
    new_mol = Chem.MolFromSmiles(new_smiles)
    new_score = Descriptors.MolLogP(new_mol) - sascorer.calculateScore(new_mol)

    res.append( (new_score - score, sim, score, new_score, smiles, new_smiles) )
    print(new_score - score, sim, score, new_score, smiles, new_smiles)

print(sum([x[0] for x in res]), sum([x[1] for x in res]))

/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: Error(s) in loading state_dict for JTPropVAE:
	size mismatch for embedding.weight: copying a param of torch.Size([780, 200]) from checkpoint, where the shape is torch.Size([780, 420]) in current model.
	size mismatch for jtnn.embedding.weight: copying a param of torch.Size([780, 200]) from checkpoint, where the shape is torch.Size([780, 420]) in current model.
	size mismatch for jtnn.W_z.weight: copying a param of torch.Size([200, 400]) from checkpoint, where the shape is torch.Size([420, 840]) in current model.
	size mismatch for jtnn.W_z.bias: copying a param of torch.Size([200]) from checkpoint, where the shape is torch.Size([420]) in current model.
	size mismatch for jtnn.W_r.weight: copying a param of torch.Size([200, 200]) from checkpoint, where the shape is torch.Size([420, 420]) in current model.
	size mismatch for jtnn.U_r.weight: copying a param of torch.Size([200, 200]) from checkpoint, where the shape is torch.Size([420, 420]) in current model.
	size mismatch 